In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
mimiciv_dir = 'C:/Users/suzie/Dropbox (MIT)/Spring 2021/6.871 Machine Learning for Healthcare/mimic-iv-1.0/hosp'

In [2]:
mimiciv_dir = 'data'

## Part I: read data

In [3]:
df_event = pd.read_csv(f'{mimiciv_dir}/diagnoses_icd.csv', low_memory=False)

In [4]:
df_code = pd.read_csv(f'{mimiciv_dir}/d_icd_diagnoses.csv', low_memory=False)

In [5]:

#merge tables:
df_sub = pd.merge(df_event, df_code, on = ['icd_code','icd_version'], how='left')

#convert to lower case:
df_sub['long_title'] = df_sub['long_title'].str.lower()

df_sub = df_sub.dropna(subset=['icd_code','icd_version','hadm_id','subject_id','long_title'], how='any')

df_sub.head(5)

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title
0,15734973,20475282,3,2825,9,sickle-cell trait
1,15734973,20475282,2,V0251,9,carrier or suspected carrier of group b strept...
2,15734973,20475282,5,V270,9,"outcome of delivery, single liveborn"
3,15734973,20475282,1,64891,9,other current conditions classifiable elsewher...
4,15734973,20475282,4,66481,9,"other specified trauma to perineum and vulva, ..."


## Part II: ICD Code Cohorting

In [6]:
#Filter to heart failure: 
#Filter 1: long title contains "heart failure" AND long title doesnt contain without heart failure
# this paper: https://academic.oup.com/aje/article/183/5/462/2462157?login=true used almost all of these for ADHF
df_f1 = df_sub[df_sub['long_title'].str.contains('heart failure')]
df_f1 = df_f1[~df_f1['long_title'].str.contains('without heart failure')]
# df_f1.head(5)

In [7]:
df_f1_hadm = df_f1[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
print(np.shape(df_f1_hadm)[0])
n1 = np.shape(df_f1_hadm)[0]
# df_f1_hadm.head(5)

64686


In [8]:
# Filter 2: Filter 1 + exclude "XX disease with heart failure" format (only keep HF as main target in title)
df_f2 = df_f1[~df_f1['long_title'].str.contains('disease')]
df_f2_hadm = df_f2[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
print(np.shape(df_f2_hadm)[0])
hf2_icd = df_f2[['long_title', 'icd_code','icd_version']].drop_duplicates(subset=['icd_code']) #.sort_values('icd_code')
hf2_icd
n2 = np.shape(df_f2_hadm)[0]

64566


In [9]:
# Filter 5: Filter 1+2+ Use I50+ and 428+
# source: https://icd.codes/icd10cm/I503 "The ICD code I50 is used to code Acute decompensated heart failure"
df_f5 = df_f2[df_f2['icd_code'].str.contains('428') | df_f2['icd_code'].str.contains('I50')]
df_f5_hadm = df_f5[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
n5=(np.shape(df_f5_hadm)[0])
n5

64409

In [10]:
# Filter 6: Filter 1+2+5+ exclude acute only, exclude chronic only, keep acute on chronic or specified
# source: NA_CARE_Prov_ICD_10_CM_Documentation_and_Coding_Best_Practices_Heart_Failure_v3_ENG_2019 pdf
df_f6 = df_f5[~df_f5['long_title'].str.contains('acute')  | df_f5['long_title'].str.contains('acute on chronic') ]
df_f6 = df_f6[~df_f6['long_title'].str.contains('chronic')  | df_f6['long_title'].str.contains('acute on chronic') ]
df_f6_hadm = df_f6[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
n6 = (np.shape(df_f6_hadm)[0])
n6

51330

In [11]:
# Filter 7: Filter 1+2+5+6 exclude end stage
# It is important to distinguish between AdHF and end-stage HF
# source https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6963179/
df_f7 = df_f6[~df_f6['long_title'].str.contains('end stage')]
df_f7_hadm = df_f7[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
n7 = (np.shape(df_f7_hadm)[0])
n7

51303

In [12]:
df_f7_hadm

,subject_id,hadm_id,long_title,icd_code,icd_version
1161,16477997,25359811,acute on chronic systolic heart failure,42823,9
1223,14959277,28740655,"congestive heart failure, unspecified",4280,9
1435,18894258,28336555,"congestive heart failure, unspecified",4280,9
1662,14974219,20480369,"congestive heart failure, unspecified",4280,9
1677,12849817,27891695,"congestive heart failure, unspecified",4280,9
1787,11508679,25781770,"congestive heart failure, unspecified",4280,9
1840,18151621,25727457,"congestive heart failure, unspecified",4280,9
2520,12711129,26466704,acute on chronic diastolic heart failure,42833,9
2622,18982551,28769191,"congestive heart failure, unspecified",4280,9
2716,13427502,20353250,"congestive heart failure, unspecified",4280,9


## Part III: DRG codes

In [18]:
drgcodes = pd.read_csv(f'{mimiciv_dir}/drgcodes.csv', low_memory=False)

In [19]:
# merge tables
df_ids_icds = pd.merge(df_event, df_code, on = ['icd_code','icd_version'], how='left')
df_merged = pd.merge(df_ids_icds, drgcodes, on = ['hadm_id', 'subject_id'], how='left')
df_merged = df_merged.drop(columns=['drg_severity', 'drg_mortality'])

In [20]:
# drop any rows that don't have a drg_code
df_merged = df_merged.dropna(subset=['drg_code'], how='any')

In [21]:
# select rows where DRG code contains "hf" or "heart failure"
df_sub = df_merged[df_merged['description'].str.contains('hf|heart failure', case=False)]
# there are some drg codes that specify "w/o hf" or "w/o heart failure" - don't include those rows
df_sub = df_sub[~df_sub['description'].str.contains('W/O AMI/HF/SHOCK|W/O AMI, HEART FAILURE OR SHOCK', case=False)]

In [22]:
# remove the hadm_ids that have a long title with heart failure in them, since the ICD codes already account for them
no_hf_icd = []
for hadm_id in df_sub.hadm_id.unique():
    df_hadm_id = df_sub[df_sub['hadm_id'] == hadm_id]
    if len(df_hadm_id[df_hadm_id['long_title'].str.contains('heart failure', case=False)]) == 0:
        no_hf_icd.append(hadm_id)
df_no_hf_icd = df_sub[df_sub['hadm_id'].isin(no_hf_icd)]

In [23]:
# only keep rows where ICD codes include:
#" cardiogenic shock", "left ventricular failure, unspecified", "shock, unspecified", or pleural effusion or fluid overload
df_no_hf_icd = df_no_hf_icd[df_no_hf_icd['long_title'].str.contains('cardiogenic shock|left ventricular failure, unspecified|shock, unspecified|Pleural effusion, not elsewhere classified|Fluid overload, unspecified', case=False)]
df_no_hf_icd_hadm = df_no_hf_icd[['subject_id','hadm_id','long_title', 'icd_code','icd_version', 'description']].drop_duplicates(subset=['hadm_id'])

## Part IV: IV Diuretics

In [79]:
chf_hadm = df_f7_hadm.hadm_id.append(df_no_hf_icd_hadm.hadm_id).unique() # 53059 hadm
len(chf_hadm)

53059

In [21]:
prescriptions = pd.read_csv(f'{mimiciv_dir}/prescriptions.csv', low_memory=False)

In [43]:
df_diuretics = pd.DataFrame({'furosemide': (prescriptions['drug'].str.contains('furosemide', case=False, na=False) | prescriptions['drug'].str.contains('lasix', case=False, na=False)) & ~prescriptions['drug'].str.contains('Desensitization', case=False, na=False), 
                   'bumetanide': (prescriptions['drug'].str.contains('bumetanide', case=False, na=False) | prescriptions['drug'].str.contains('bumex', case=False, na=False)) & ~prescriptions['drug'].str.contains('Desensitization', case=False, na=False), 
                   'chlorothiazide': (prescriptions['drug'].str.contains('chlorothiazide', case=False, na=False) | prescriptions['drug'].str.contains('diuril', case=False, na=False)) & ~prescriptions['drug'].str.contains('Desensitization', case=False, na=False) & ~prescriptions['drug'].str.contains('hydrochlorothiazide', case=False, na=False)})

In [44]:
# prescriptions for diuretics
given_diuretic = df_diuretics.any(axis=1)

In [80]:
# prescriptions with hadm_id associated w heart failure ICD
in_icd = np.isin(prescriptions.hadm_id, chf_hadm)

In [81]:
# prescriptions given via IV
route_iv = prescriptions.route.str.contains('IV', case=False, na=False)

In [82]:
# final cohort with all prescription fields
cohort = prescriptions[given_diuretic & in_icd & route_iv]

In [83]:
# unique drug names to confirm no unwanted drugs are included
cohort.drug.unique()

array(['Furosemide', 'Chlorothiazide Sodium', 'Furosemide-Heart Failure',
       'Chlorothiazide', 'Bumetanide', 'Furosemide (Latex Free)', 'LaSIX',
       'bumex'], dtype=object)

In [84]:
cohort.hadm_id.unique().shape[0] #27129 unique HADM_IDs

27135

In [85]:
cohort.subject_id.unique().shape[0] #15875 unique SUBJECT_IDs

15879

In [86]:
cohort_hadm = cohort.hadm_id.unique()

## Part V: Removing Dialysis Patients 

In [100]:
dialysis_codes = ["Z992", "N186", "5856", "V4511"]

In [101]:
df_event["chronic_dialysis"] = df_event["icd_code"].str.strip().isin(dialysis_codes)

In [105]:
df_event

,subject_id,hadm_id,seq_num,icd_code,icd_version,chronic_dialysis
0,15734973,20475282,3,2825,9,False
1,15734973,20475282,2,V0251,9,False
2,15734973,20475282,5,V270,9,False
3,15734973,20475282,1,64891,9,False
4,15734973,20475282,4,66481,9,False
...,...,...,...,...,...,...
5280346,13747041,25594844,6,R531,10,False
5280347,13747041,25594844,8,R0902,10,False
5280348,13747041,25594844,4,F1120,10,False
5280349,13747041,25594844,2,J189,10,False


## Part VI: Final Cohort

In [117]:
final_cohort = df_event.drop(columns=['seq_num', 'icd_code', 'icd_version'])
final_cohort.head()

,subject_id,hadm_id,chronic_dialysis
0,15734973,20475282,False
1,15734973,20475282,False
2,15734973,20475282,False
3,15734973,20475282,False
4,15734973,20475282,False


In [118]:
final_cohort['CHF exacerbation'] = final_cohort['hadm_id'].isin(cohort_hadm)

In [124]:
final_cohort = final_cohort.drop_duplicates()

In [125]:
final_cohort.head()

,subject_id,hadm_id,chronic_dialysis,CHF exacerbation
0,15734973,20475282,False,False
5,11442057,21518990,False,False
11,10072949,20817034,False,False
16,13538405,24822466,False,False
21,10287061,27485182,False,False


In [127]:
len(final_cohort)

536165

In [126]:
final_cohort.to_csv('final_cohort.csv')